In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import numpy as np
import math

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [ ]:
config_file = '../testingdir/parameters.xml'

cfg = Parameters(config_file)

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
shape=[0, 0, cfg.number_r_grid, cfg.number_z_grid]

use_grid=True
ylim=None
use_cache=False
verbose=True

specie='Electrons'
time_range=[cfg.start_time, cfg.end_time]

autoselect = True
show_grid=False

x_axis_label = r'$\mathit{t (s)}$'
y_axis_label = r'$\mathit{T (eV)}$'

plot_name = r'$\mathbf{Temperature-time \enspace Dependency}\enspace(T)$'

In [ ]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         data_root=cfg.data_root,
                         fullframe_size=[cfg.number_r_grid , cfg.number_z_grid],
                         fpds=cfg.frames_per_file, 
                         use_cache=use_cache,
                         verbose=True)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True


In [ ]:
# get data
data = []
data_collector = []

for probe in cfg.probes:
    start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
    end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
    if (probe.type == 'frame') and (probe.specie == specie) and (probe.r_start == shape[0]) and (probe.z_start == shape[1]) and(probe.r_end == shape[2]) and(probe.z_end == shape[3]) and (probe.component == 'T'):
        data_collector = reader.get_frame_range('T/{}'.format(probe.specie), shape, start_frame, end_frame)
        for i in range(0, len(data_collector)):
            a = data_collector[i].flatten()
            index = np.argwhere(np.isnan(a))
            b = np.delete(a, index)
            data.append(np.sum(b) / len(b))
            

# try bigger frames, if autoselect enabled
if len(data) == 0 and autoselect:
    for probe in cfg.probes:
        probe_shape = [probe.r_start, probe.z_start, probe.r_end, probe.z_end]
        start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
        end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
        if (probe.type == 'frame') and (probe.specie == specie) and (probe_shape[0] <= shape[0]) and (probe_shape[1] <= shape[1]) and(probe_shape[2] >= shape[2]) and(probe_shape[3] >= shape[3]) and (probe.component == 'T'):
            for piece in range(start_frame, end_frame, cfg.frames_per_file):
                data_collector[piece - start_frame:piece - start_frame + cfg.frames_per_file - 1] = reader.get_frame_range('T/{}'.format(probe.specie), probe_shape, piece, piece + cfg.frames_per_file)[:, shape[0]:shape[2], shape[1]:shape[3]]

            for i in range(0, len(data_collector)):
                a = data_collector[i].flatten()
                index = np.argwhere(np.isnan(a))
                b = np.delete(a, index)
                data.append(np.sum(b) / len(b))
data = np.nan_to_num(data)

In [ ]:
data_timeline = np.linspace(time_range[0], time_range[1], len(data))

# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=use_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   x_ticklabel_start=time_range[0],
                   x_ticklabel_end=time_range[1]
                  )

# add subplots
plot_t = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# set y-limits
if ylim is not None:
    plot_t.set_ylim(ylim)

In [ ]:
plot_t.plot(data_timeline, data)

plot.show()